# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import pandas as pd
pd.set_option('display.max_columns', 100)
import os
import re
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sqlalchemy import create_engine

In [ ]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse_table', engine)

# display loaded dataframe
df

In [ ]:
# have a quick overview on the dataset
df.describe()

In [ ]:
# a quick check on the max value for each columns
df.max()

In [ ]:
# a quick check on the min value for each columns
df.min()

In [ ]:
# we remove 'child_alone' column as it has only 0 (ZERO) values.
df = df.drop('child_alone', axis = 1)

In [ ]:
# maximum value for 'related' column is '2'. Let us investigate further...
df.groupby('related').count()

### Note:
From the results above, '2' could be error. We can replace it with '0' or '1'. In this case, I will replace it with '1'

In [ ]:
df['related'] = df['related'].map(lambda x: 1 if x==2 else x)      

In [ ]:
# check if all '2' have been replace with '1'
df.groupby('related').count()

In [ ]:
# separate the dataset to input variables (X) and target variables (y)
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### Implement the StartingVerbExtractor class

Create custom transformer to be used for FeatureUnion later in the ML pipeline.

In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
def ML_pipeline(clf = AdaBoostClassifier()):
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),
        ('clf', MultiOutputClassifier(clf))
    ])
    
    return pipeline

# include StartingVerbExtractor custom transformer
def ML_pipeline_2(clf = AdaBoostClassifier()):
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(clf))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = ML_pipeline()
model.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred_test = model.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

### 6. Improve your model

Use grid search to find better parameters. 

In [ ]:
# get list of model parameters
model.get_params().keys()

In [ ]:
model_2 = ML_pipeline()

# Model training using GridSearchCV is computational extensive task. 
# Computational time increases as we increase the number of parameters.
# In view of that, only a few parameters are chosen to demonstrate the concept.
# To obtain a highly optimized model, we need to increase the number of parameters.

parameters = {
    'clf__estimator__learning_rate': [0.5, 1.0],
    'clf__estimator__n_estimators': [10, 20]
}


cv = GridSearchCV(model_2, param_grid=parameters, cv=5, n_jobs=-1, verbose=3) 
# verbose=3 to get real time training progress
# n_jobs=-1 -> to train in parallel across the maximum number of cores in our computer, spending it up.
# cv=5 -> 5-fold cross validation


cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_test = cv.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# try other model -> RandomForestClassifier
rf_model = ML_pipeline(clf = RandomForestClassifier())
rf_model.fit(X_train, y_train)
y_pred_rf_test = rf_model.predict(X_test)

print(classification_report(y_test.values, y_pred_rf_test, target_names=y.columns.values))

In [ ]:
# we can also try using ML_pipeline_2 that includes custom transformer of 'StartingVerbEstimator'
model_3 = ML_pipeline_2()
model_3.fit(X_train, y_train)
y_pred_3_test = model_3.predict(X_test)

print(classification_report(y_test.values, y_pred_3_test, target_names=y.columns.values))

### 9. Export your model as a pickle file

In [ ]:
# save model in pickle file
with open('classifier.pkl', 'wb') as f:
    pickle.dump(model_3, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.